# Traditional Methods trained on every file, tuned on the rest (untuned)
MinMaxScaler is applied to these tests. All models are untuned. The model will be trained on file A, tested on files B,C,D.., then on B, tested on A,C,D.. etc

In [1]:
import sys
sys.path.append("../../")
from time import process_time
from os import listdir, chdir, environ

def warn(*args, **kwargs):
  pass
import warnings
warnings.warn = warn

if not sys.warnoptions:
    warnings.simplefilter("ignore")
    environ["PYTHONWARNINGS"] = "ignore"

import numpy as np
from modules.NetworkTraffic import NT2
from sklearn import model_selection, metrics
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import make_pipeline

from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import LinearSVC, SVC
from sklearn.neural_network import MLPClassifier

FilesToTest = list()
chdir("../../data")
for file in listdir():
  if file.endswith(".csv"):
    FilesToTest.append(file)

TestSize = [0.4]
ModelsToTest = [RandomForestClassifier(), GradientBoostingClassifier(), DecisionTreeClassifier(), MLPClassifier(), LinearSVC()]
OutputResults = dict()
#ModelResults = dict()

In [2]:
from copy import deepcopy

def testMe():
  OutputResults.clear()

  for index, file in enumerate(FilesToTest):
    print(file, end=', ')
    OutputResults[file] = dict()
    currentFileData = NT2(file, transform=True, drop=True)
    restOfFiles = deepcopy(FilesToTest)
    restOfFiles.pop(index)
    restOfTheFilesData = NT2(restOfFiles, transform=True, drop=True)

    for model in ModelsToTest:
      #print(f"{file} : {str(model)}...")
      clf = make_pipeline(MinMaxScaler(), model)
      clf.fit(currentFileData.data, currentFileData.target)
      y_pred = clf.predict(restOfTheFilesData.data)
      score = metrics.accuracy_score(restOfTheFilesData.target, y_pred)
      OutputResults[file].update({str(model): {"Accuracy": score}})

In [3]:
import json
from os import path

def writeMe():
  with open("EveryFileTransfer_Untuned_AllTestResults.json", "a") as f:
    f.write(json.dumps(OutputResults, indent=2))

  with open("EveryFileTransfer_Untuned_ModelResults.csv", "a") as f3:
    if not path.exists("EveryFileTransfer_Untuned_ModelResults.csv"): f3.write("File Trained On,Model,Accuracy\n")
    for file in OutputResults:
      for model in OutputResults[file]:
        f3.write(f"{file},{model},{OutputResults[file][model]['Accuracy']}\n")

In [4]:
REPEATS = 5
for _ in range(0, REPEATS):
  testMe()
  writeMe()

b1000d10.csv, b1000d100.csv, b1000d30.csv, b100d10.csv, b100d100.csv, b100d30.csv, b5000d10.csv, b5000d100.csv, b5000d30.csv, b1000d10.csv, b1000d100.csv, b1000d30.csv, b100d10.csv, b100d100.csv, b100d30.csv, b5000d10.csv, b5000d100.csv, b5000d30.csv, b1000d10.csv, b1000d100.csv, b1000d30.csv, b100d10.csv, b100d100.csv, b100d30.csv, b5000d10.csv, b5000d100.csv, b5000d30.csv, b1000d10.csv, b1000d100.csv, b1000d30.csv, b100d10.csv, b100d100.csv, b100d30.csv, b5000d10.csv, b5000d100.csv, b5000d30.csv, b1000d10.csv, b1000d100.csv, b1000d30.csv, b100d10.csv, b100d100.csv, b100d30.csv, b5000d10.csv, b5000d100.csv, b5000d30.csv, 